In [4]:
import cv2 
import os
from PIL import Image
import numpy as np

In [13]:
def findK(percentClarity, arr):
    k = 0
    s = 0
    arrSum = sum(arr)
    for i in arr:
        s += i
        p = (s/arrSum)*100
        k+=1
        if round(p) >= percentClarity:
            return k

def compress_channel(img_array, channel, percentage):
    specific_channel = img_array[:,:,channel]
    U, S, Vt = np.linalg.svd(specific_channel)
    k = findK(percentage, S)
    reconstructed_channel = U[:,:k] @ np.diag(S[:k]) @ Vt[:k,:]
    return reconstructed_channel

def compressed_image(img_array, percentage):
    l, w, h = img_array.shape
    newimg_array = np.zeros((l,w,h))
    for channel in range(h):
        newimg_array[:,:,channel] = compress_channel(img_array, channel, percentage)
    return newimg_array

In [14]:
# Function to extract frames

def extract_frames(vid_path):

    fps = 25.0
    size = (100, 100)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    videoName = vid_path.split("/")[-1].split(".mp4")[0]

    output_path = 'output_videos/' + videoName + '.mp4'

    # Path to video file
    videoObj = cv2.VideoCapture(vid_path)

    # checks whether frames were extracted
    success = 1
    out = cv2.VideoWriter(output_path, fourcc, fps, size)
    while success:
        success, image = videoObj.read()
        if image is not None:
            compressedImage = compressed_image(image, 100)
            out.write(compressedImage)
    out.release()
    return videoName

In [20]:
p = r'C:/Harrithha/IIT_Dh/College_Academics/Sem_6/MDS/MDS_2023/Assignment_1/images/1.png'
img = Image.open(p)
print(img.shape)
newImg = compressed_image(img, 100)
i = Image.fromarray(newImg)
display(i)


(100, 100, 4)


TypeError: Cannot handle this data type: (1, 1, 4), <f8

In [8]:
directory = r"C:/Harrithha/IIT_Dh/College_Academics/Sem_6/MDS/MDS_2023/Assignment_1/videos/"

for video in os.listdir(directory):
    videoPath  = directory + video
    vidName = extract_frames(videoPath)

<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'numpy.ndarray'>
(100, 100, 3)
<class 'nump

In [7]:
def createVideo(path, name):

    # Define the frame rate and video size
    fps = 25.0
    size = (100, 100)

    # Define the codec to use for the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Define the output file name and path
    output_path = 'output_videos/' + name + '.mp4'

    # Create a new video writer object
    out = cv2.VideoWriter(output_path, fourcc, fps, size)

    # Loop through the images in the folder and add them to the video
    for filename in sorted(os.listdir(path)):
        if filename.endswith('.jpg'):
            img_path = os.path.join(path, filename)
            img = 
            print(type(img))
            print(img.shape)
            out.write(img)

    # Release the video writer object and close the video file
    out.release()

SyntaxError: invalid syntax (1443858597.py, line 20)